In [6]:
import os
import subprocess

from Bio.PDB import PDBParser
from tqdm.notebook import tqdm
from vina import Vina

ModuleNotFoundError: No module named 'vina'

In [ ]:
parser = PDBParser(QUIET = True)
parser.get_structure("", "receptor/botA_LC.pdb")

In [ ]:
center = [] # 16.538, -9.532, 23.655
box_size = [20, 20, 20]

In [ ]:
filenames = [filename for filename in os.listdir("ligands") if os.path.splitext(filename)[1] == ".mol2"]
ligand_names = [os.path.splitext(filename)[0] for filename in filenames]

In [ ]:
for i, ligand_name in tqdm(enumerate(ligand_names)):
    cmd = f"mk_prepare_ligand.py -i ligands/{ligand_name}.mol2 -o ligands/{ligand_name}.pdbqt"
    subprocess.run(cmd, shell=True)

In [ ]:
vina = Vina()
affinities = {}
for i, ligand_name in tqdm(enumerate(ligand_names)):
    start_time = time.time()
    vina.set_receptor("receptor/botA_LC.pdbqt")
    vina.set_ligand_from_file(f"ligands/{ligand_name}.pdbqt")
    vina.compute_vina_maps(center, box_size)
    vina.dock(exhaustiveness=10, n_poses=9)
    vina.write_poses(f"{ligand_name}_docked.pdbqt", n_poses=1, overwrite=True)
    affinities[ligand_name] = vina.energies()[0, 0]

#ZINC000004534051